# Analysing the Hotel Rates (2004–2018)

In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
# Load the csv file as a dataframe
hotelrates_df = pd.read_csv("Data/hotel_dailyrates.csv", encoding = "utf-8")
hotelrates_df.head()

,Average Daily Rate,Hotel Occupancy,Date
0,141.6038,0.738484,2004-07
1,143.1187,0.719571,2004-08
2,137.4212,0.750757,2004-09
3,141.5987,0.778466,2004-10
4,148.4632,0.682347,2004-11


## Converting average daily rates to monthly rates
- Create subsets for months that don't change number of days
- Create a subset for February (which has different days depending on the year)

In [3]:
# Create a list of years from the Date column
years_all = [date[0:4] for date in hotelrates_df["Date"]] 
    # date[0:4] gives substrings from the Date column
    
years_all = [int(i) for i in years_all] # convert list of strings to list of int

### Regular months (30 and 31 days)

In [4]:
# Create a list of months in numerical format
month_30 = ["04","06","09","11"]
month_31 = ["01","03","05","07","08","10","12"]
month_28 = ["02"]

month_by_day = [month_30, month_31]
days = [30, 31]

In [5]:
# Define a function that extracts the month (2-digit format) from the Date column
def months(df,list):
    x = [df.loc[df["Date"].str.slice(-2) == item] for item in list]
    return x

In [6]:
# Create lists for months with 30 and 31 days whatever the year
month_grps = [months(hotelrates_df,grp) for grp in month_by_day] # groups of months based on number of days

avg_daily = [[(month["Average Daily Rate"]) for month in grp] for grp in month_grps] # daily average rates per group
mo_rate_30 = [(i * days[0]) for i in avg_daily[0]] # monthly rates in the 30-day group       
mo_rate_31 = [(i * days[1]) for i in avg_daily[1]] # monthly rates in the 31-day group

In [7]:
# Create dataframes for months grouped by number of days (30,31)
df30 = pd.concat(month_grps[0]) # puts all values in the same columns
df30["Monthly Rate"] = pd.concat(mo_rate_30) # add the monthly rate as an additional column

df31 = pd.concat(month_grps[1]) # puts all values in the same columns
df31["Monthly Rate"] = pd.concat(mo_rate_31) # add the monthly rate as an additional column

df31.head()

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
6,148.8121,0.729100,2005-01,4613.1751
18,160.7223,0.755878,2006-01,4982.3913
30,179.4341,0.727883,2007-01,5562.4571
42,184.5935,0.785091,2008-01,5722.3985
54,169.4801,0.742489,2009-01,5253.8831


### For February, if the year falls on a leap year, the number of days is 29. If not, 28.

In [8]:
# Extract a dataframe that contains the data for Feb
Feb = months(hotelrates_df,month_28)
Feb = Feb[0]

In [9]:
# Define a function that creates a list of leap years 
# Resource: https://stackoverflow.com/a/18951699/10205963
def leap(start,end):
    return [year for year in range(start, end + 1)
            if (year % 400 == 0) or (year % 4 == 0 and not year % 100 == 0)]

In [10]:
# Get the leap years from the list of years
leap_yrs = leap(years_all[0], years_all[-1]) # years[0] = 1st; years[-1] = last
leap_yrs = [str(i) for i in leap_yrs]
leap_yrs

['2004', '2008', '2012', '2016']

In [11]:
# Create a subset for Feb data for leap years
leap_Feb = Feb.loc[Feb["Date"].str.contains("|".join(leap_yrs))]

# Calculate monthly rates for Feb leap years
leap_Feb_mo_rate = [(i * 29) for i in leap_Feb["Average Daily Rate"]] 

# Add the Monthly Rates as a new column
# Resource: https://stackoverflow.com/a/47694287/10205963

se = pd.Series(leap_Feb_mo_rate) # Convert the monthly rate list to a series
leap_Feb.insert(loc = 3, column = "Monthly Rate", value = se.values) # add the column Monthly Rate (no errors)
    
leap_Feb

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
43,197.1489,0.811044,2008-02,5717.3181
91,213.3869,0.822019,2012-02,6188.2201
139,315.3700,0.935458,2016-02,9145.7300


In [12]:
# Create a subset for Feb data for regular years
reg_Feb = Feb.loc[~Feb["Date"].str.contains("|".join(leap_yrs))]

# Calculate monthly rates for Feb regular years
reg_Feb_mo_rate = [(i * 28) for i in reg_Feb["Average Daily Rate"]] 


# Add the Monthly Rates as a new column

se2 = pd.Series(reg_Feb_mo_rate) # Convert the monthly rate list to a series
reg_Feb.insert(loc = 3, column = "Monthly Rate", value = se2.values) # add the column Monthly Rate (no errors)

reg_Feb

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
7,151.1379,0.740805,2005-02,4231.8612
19,161.3353,0.723245,2006-02,4517.3884
31,186.9618,0.791289,2007-02,5234.9304
55,157.9055,0.648616,2009-02,4421.3540
67,150.5339,0.762755,2010-02,4214.9492
79,181.8734,0.837384,2011-02,5092.4552
103,201.8483,0.854944,2013-02,5651.7524
115,234.2935,0.898843,2014-02,6560.2180
127,236.7917,0.883478,2015-02,6630.1676
151,292.0611,0.914510,2017-02,8177.7108


In [13]:
# Concatenate the two dataframes together (Feb)
dfFeb = pd.concat([leap_Feb,reg_Feb])
dfFeb

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
43,197.1489,0.811044,2008-02,5717.3181
91,213.3869,0.822019,2012-02,6188.2201
139,315.3700,0.935458,2016-02,9145.7300
7,151.1379,0.740805,2005-02,4231.8612
19,161.3353,0.723245,2006-02,4517.3884
31,186.9618,0.791289,2007-02,5234.9304
55,157.9055,0.648616,2009-02,4421.3540
67,150.5339,0.762755,2010-02,4214.9492
79,181.8734,0.837384,2011-02,5092.4552
103,201.8483,0.854944,2013-02,5651.7524


In [14]:
# Concatenate all dataframes (30-day, 31-day, Feb)
hotelrates_df2 = pd.concat([df30, df31, dfFeb])
hotelrates_df2.head()

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate
9,157.8543,0.740657,2005-04,4735.629
21,169.9458,0.770442,2006-04,5098.374
33,183.4274,0.764485,2007-04,5502.822
45,183.7442,0.768456,2008-04,5512.326
57,157.9462,0.743636,2009-04,4738.386


In [15]:
# Correct the monthly hotel rates based on the hotel occupancy rates
se3 = pd.Series(round((hotelrates_df2["Monthly Rate"] * hotelrates_df2["Hotel Occupancy"]),3))
hotelrates_df2.insert(loc = 4, column = "Corrected Monthly Rate", value = se3.values)

hotelrates_df2.head()

,Average Daily Rate,Hotel Occupancy,Date,Monthly Rate,Corrected Monthly Rate
9,157.8543,0.740657,2005-04,4735.629,3507.477
21,169.9458,0.770442,2006-04,5098.374,3928.002
33,183.4274,0.764485,2007-04,5502.822,4206.826
45,183.7442,0.768456,2008-04,5512.326,4235.983
57,157.9462,0.743636,2009-04,4738.386,3523.632


In [16]:
# Save dataframe as csv file
hotelrates_df2.to_csv("Data/hotel_monthlyrate.csv", encoding = "utf-8", index = False)

### Calculate yearly hotel rates

In [17]:
# Convert the years_all list into a list of strings
years_all_str = set([str(i) for i in years_all]) # unique years
years_all_str = list(years_all_str) # pass the set as a list (for list comprehension below)
years_all_int = [int(i) for i in years_all_str] # convert to int (for graphing)
years_all_int

[2006,
 2014,
 2011,
 2009,
 2004,
 2010,
 2008,
 2015,
 2013,
 2012,
 2016,
 2017,
 2007,
 2005,
 2018]

In [18]:
# Define a function that returns the sum of each year
def totals(df,str):
    data = df[df["Date"].str.contains(str)]
    year_rate = data["Corrected Monthly Rate"].sum()
    return year_rate

In [19]:
yearly_rates = [round(totals(hotelrates_df2,yr),3) for yr in years_all_str]

In [20]:
# Create a new dataframe 
hotel_year_rates = pd.DataFrame(dict(years_all_int = years_all_int,
                                     yearly_rates = yearly_rates))
hotel_year_rates = hotel_year_rates.rename(columns = {"years_all_int": "Year",
                                                      "yearly_rates": "Yearly Rate"})
hotel_year_rates.head()

,Year,Yearly Rate
0,2006,46488.121
1,2014,79795.282
2,2011,55845.469
3,2009,43366.102
4,2004,19596.943


In [21]:
# Save dataframe as csv file
hotel_year_rates.to_csv("Data/hotel_yrrate.csv", encoding = "utf-8", index = False)